<a href="https://colab.research.google.com/github/rumeshsmrr/reinforcement-learning-lunarlander/blob/main/Gymnasium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!apt-get install -y swig
!pip install gymnasium[box2d] stable-baselines3[extra]


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [29]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from gymnasium.wrappers import RecordVideo
import os
from IPython.display import HTML
from base64 import b64encode


In [30]:
env = gym.make("LunarLander-v3")


In [31]:
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=2_000_000)


Streaming output truncated to the last 5000 lines.
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.415    |
|    n_updates        | 183432   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1e+03    |
|    ep_rew_mean      | -13      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1572     |
|    fps              | 811      |
|    time_elapsed     | 909      |
|    total_timesteps  | 737830   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.383    |
|    n_updates        | 184432   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1e+03    |
|    ep_rew_mean      | -13.2    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1576     |
|   

In [32]:
model.save("dqn_lunarlander")
del model  # delete the model to demonstrate loading

model = DQN.load("dqn_lunarlander", env=env)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [33]:
from gymnasium.wrappers import RecordVideo
import os

video_folder = "./videos"
os.makedirs(video_folder, exist_ok=True)

env = gym.make("LunarLander-v3", render_mode="rgb_array")
env = RecordVideo(env, video_folder, episode_trigger=lambda e: True)

obs, _ = env.reset()
done = False
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

env.close()


/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [34]:
from IPython.display import HTML
from base64 import b64encode

video_path = sorted(os.listdir(video_folder))[-1]
video_path = os.path.join(video_folder, video_path)
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")
